In [2]:
import pandas as pd

In [5]:
my_books = pd.read_csv("liked_books.csv", index_col=0)

In [10]:
my_books.describe()

,user_id,book_id,rating
count,27.0,2.700000e+01,27.000000
mean,-1.0,4.105047e+06,4.407407
std,0.0,6.814056e+06,1.448056
min,-1.0,1.898000e+03,1.000000
25%,-1.0,9.808750e+04,5.000000
50%,-1.0,4.371430e+05,5.000000
75%,-1.0,5.337486e+06,5.000000
max,-1.0,2.565945e+07,5.000000


In [11]:
my_books["book_id"] = my_books["book_id"].astype(str)

In [12]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id ] = book_id

In [13]:
book_set = set(my_books["book_id"])

In [16]:
overlap_users ={}


with open("goodreads_interactions.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        user_id, csv_id, _ , rating, _ = line.strip().split(",")

        book_id = csv_book_mapping.get(csv_id)
        if book_id in book_set:
            if user_id not in overlap_users:
                overlap_users[user_id] = 1
            else:
                overlap_users[user_id] += 1


In [18]:
overlap_users

{'0': 3,
 '1': 1,
 '2': 2,
 '3': 2,
 '5': 1,
 '7': 1,
 '9': 1,
 '10': 1,
 '11': 1,
 '12': 1,
 '15': 3,
 '16': 1,
 '18': 2,
 '20': 1,
 '22': 3,
 '23': 1,
 '26': 2,
 '29': 2,
 '30': 1,
 '31': 2,
 '32': 2,
 '33': 3,
 '34': 1,
 '38': 3,
 '39': 2,
 '40': 1,
 '41': 2,
 '42': 1,
 '43': 1,
 '44': 1,
 '47': 2,
 '50': 1,
 '51': 3,
 '52': 2,
 '60': 1,
 '61': 1,
 '63': 1,
 '66': 3,
 '68': 3,
 '72': 1,
 '73': 1,
 '77': 1,
 '78': 2,
 '79': 3,
 '80': 3,
 '81': 2,
 '84': 1,
 '85': 1,
 '87': 2,
 '89': 1,
 '90': 4,
 '92': 1,
 '98': 2,
 '102': 1,
 '104': 2,
 '106': 2,
 '108': 1,
 '109': 1,
 '110': 1,
 '113': 2,
 '119': 1,
 '121': 1,
 '124': 3,
 '125': 1,
 '126': 2,
 '127': 1,
 '132': 3,
 '133': 1,
 '136': 3,
 '137': 2,
 '138': 1,
 '139': 2,
 '140': 1,
 '145': 2,
 '148': 2,
 '149': 1,
 '150': 2,
 '151': 1,
 '155': 2,
 '156': 1,
 '157': 2,
 '158': 1,
 '159': 1,
 '163': 1,
 '165': 1,
 '166': 2,
 '167': 5,
 '168': 2,
 '170': 1,
 '172': 3,
 '173': 1,
 '174': 2,
 '176': 1,
 '177': 1,
 '179': 1,
 '180': 3,
 '18

In [17]:
len(overlap_users)

316341

In [19]:
filtered_overlap_users = set([k for k in overlap_users if overlap_users[k] > my_books.shape[0]/5])

In [21]:
len(filtered_overlap_users)

1258

In [22]:
interactions_list = []

with open("goodreads_interactions.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        user_id, csv_id, _ , rating, _ = line.strip().split(",")

        if user_id in filtered_overlap_users:
            book_id = csv_book_mapping[csv_id]
            interactions_list.append([user_id, book_id, rating])


In [23]:
len(interactions_list)

5638701

In [24]:
interactions_list[0]

['282', '627206', '4']

In [27]:
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "rating"])

In [28]:
interactions = pd.concat([my_books[["user_id", "book_id", "rating"]], interactions])

In [29]:
interactions

,user_id,book_id,rating
0,-1,2517439,5
1,-1,113576,5
2,-1,35100,5
3,-1,228221,5
5,-1,17662739,5
...,...,...,...
5638696,804100,475178,0
5638697,804100,186074,0
5638698,804100,153008,0
5638699,804100,45107,0


In [30]:
interactions["book_id"] = interactions["book_id"].astype(str)
interactions["user_id"] = interactions["user_id"].astype(str)
interactions["rating"] = pd.to_numeric(interactions["rating"])

In [31]:
interactions["user_id"].unique()

array(['-1', '282', '874', ..., '442043', '712588', '804100'],
      dtype=object)

In [32]:
interactions["user_index"] = interactions["user_id"].astype("category").cat.codes

In [39]:
len(interactions["user_index"].unique())

1259

In [37]:
interactions["book_index"] = interactions["book_id"].astype("category").cat.codes

In [42]:
len(interactions["book_index"].unique())

802870

In [43]:
from scipy.sparse import coo_matrix

ratings_mat_coo = coo_matrix((interactions["rating"], (interactions["user_index"], interactions["book_index"])))

In [44]:
ratings_mat_coo

<COOrdinate sparse matrix of dtype 'int64'
	with 5638728 stored elements and shape (1259, 802870)>

In [46]:
ratings_mat = ratings_mat_coo.tocsr()

In [47]:
ratings_mat

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 5638728 stored elements and shape (1259, 802870)>

In [48]:
interactions[interactions["user_id"]=="-1"]

,user_id,book_id,rating,user_index,book_index
0,-1,2517439,5,0,414880
1,-1,113576,5,0,38971
2,-1,35100,5,0,575858
3,-1,228221,5,0,356004
5,-1,17662739,5,0,214285
6,-1,356824,5,0,581743
7,-1,12125412,5,0,59763
8,-1,139069,5,0,124430
10,-1,76680,5,0,722098
11,-1,1898,5,0,276178


In [49]:
my_index = 0

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index, :], ratings_mat).flatten()

In [53]:
similarity

array([1.        , 0.04579826, 0.06143443, ..., 0.        , 0.00393254,
       0.02317069])

In [86]:
import numpy as np

indices = np.argpartition(similarity, -100)[-100:]

In [87]:
indices

array([ 407,  479,  704,  946,  436, 1162,  682,  129,  276,  738,  126,
        125, 1163,  807,  435,  995,  684,  426,  897, 1178,  806, 1180,
        153,  672,  916,  892,   96,  687, 1143, 1188,  826,  992,  922,
         88,  660,  321,  776,  781,  927,  287, 1045,  394,  878,   76,
        485, 1204,  392, 1118,  771, 1208, 1111, 1210,  695,  493, 1213,
        942,  379,  495,  496,   59, 1106,  190,  862,  767,   51,  374,
        193,  953,   47, 1001,  795,  195,  638,  794,  294, 1088,  295,
       1087, 1021,  210,  211,  218,  356,  964,  332,   19,  969,  711,
        792,  972, 1249,  540, 1251, 1252,  608,  239, 1255,    6,    2,
          0], dtype=int64)

In [88]:
similar_users = interactions[interactions["user_index"].isin(indices)].copy()

In [89]:
similar_users

,user_id,book_id,rating,user_index,book_index
0,-1,2517439,5,0,414880
1,-1,113576,5,0,38971
2,-1,35100,5,0,575858
3,-1,228221,5,0,356004
5,-1,17662739,5,0,214285
...,...,...,...,...,...
5638521,712588,32388712,3,1143,543119
5638522,712588,16322,5,1143,183365
5638523,712588,860543,0,1143,759827
5638524,712588,853510,5,1143,756768


In [90]:
similar_users = similar_users[similar_users["user_id"] != "-1"]

In [91]:
similar_users

,user_id,book_id,rating,user_index,book_index
14402,1625,11084145,0,190,30490
14403,1625,6493208,0,190,674555
14404,1625,9742,0,190,794964
14405,1625,49195,0,190,621570
14406,1625,43015,3,190,605901
...,...,...,...,...,...
5638521,712588,32388712,3,1143,543119
5638522,712588,16322,5,1143,183365
5638523,712588,860543,0,1143,759827
5638524,712588,853510,5,1143,756768


In [92]:
book_recs = similar_users.groupby("book_id").rating.agg(['count', 'mean'])

In [93]:
book_recs

,count,mean
book_id,,
1,69,4.405797
10,1,5.000000
1000303,1,0.000000
100065,1,0.000000
1000751,2,1.500000
...,...,...
9998,2,0.000000
9998825,1,4.000000
9999,1,0.000000


In [94]:
books_titles = pd.read_json("books_titles.json")

books_titles["book_id"] = books_titles["book_id"].astype(str)

In [95]:
book_recs = book_recs.merge(books_titles, how="inner", on="book_id")

In [96]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title
0,1,69,4.405797,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,harry potter and the halfblood prince harry po...
1,10,1,5.000000,"Harry Potter Collection (Harry Potter, #1-6)",25245,https://www.goodreads.com/book/show/10.Harry_P...,https://images.gr-assets.com/books/1328867351m...,harry potter collection harry potter 16
2,1000303,1,0.000000,John Tyler: The Accidental President,874,https://www.goodreads.com/book/show/1000303.Jo...,https://s.gr-assets.com/assets/nophoto/book/11...,john tyler the accidental president
3,100065,1,0.000000,As a Driven Leaf,2047,https://www.goodreads.com/book/show/100065.As_...,https://s.gr-assets.com/assets/nophoto/book/11...,as a driven leaf
4,1000751,2,1.500000,"Pollyanna (Pollyanna, #1)",58040,https://www.goodreads.com/book/show/1000751.Po...,https://s.gr-assets.com/assets/nophoto/book/11...,pollyanna pollyanna 1
...,...,...,...,...,...,...,...,...
24154,9998,2,0.000000,The Woman in the Dunes,11841,https://www.goodreads.com/book/show/9998.The_W...,https://images.gr-assets.com/books/1361254930m...,the woman in the dunes
24155,9998825,1,4.000000,"Northwest Angle (Cork O'Connor, #11)",3357,https://www.goodreads.com/book/show/9998825-no...,https://s.gr-assets.com/assets/nophoto/book/11...,northwest angle cork oconnor 11
24156,9999,1,0.000000,The Box Man,2791,https://www.goodreads.com/book/show/9999.The_B...,https://images.gr-assets.com/books/1320459929m...,the box man
24157,9999107,2,1.000000,The American Heiress,24522,https://www.goodreads.com/book/show/9999107-th...,https://images.gr-assets.com/books/1307342832m...,the american heiress


In [97]:
book_recs["adjusted_count"] = book_recs["count"] * (book_recs["count"] / book_recs["ratings"])

In [98]:
book_recs["score"] = book_recs["mean"] * book_recs["adjusted_count"]

In [100]:
book_recs = book_recs[~book_recs["book_id"].isin(my_books["book_id"])]

In [101]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
0,1,69,4.405797,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,harry potter and the halfblood prince harry po...,0.002778,0.012239
1,10,1,5.000000,"Harry Potter Collection (Harry Potter, #1-6)",25245,https://www.goodreads.com/book/show/10.Harry_P...,https://images.gr-assets.com/books/1328867351m...,harry potter collection harry potter 16,0.000040,0.000198
2,1000303,1,0.000000,John Tyler: The Accidental President,874,https://www.goodreads.com/book/show/1000303.Jo...,https://s.gr-assets.com/assets/nophoto/book/11...,john tyler the accidental president,0.001144,0.000000
3,100065,1,0.000000,As a Driven Leaf,2047,https://www.goodreads.com/book/show/100065.As_...,https://s.gr-assets.com/assets/nophoto/book/11...,as a driven leaf,0.000489,0.000000
4,1000751,2,1.500000,"Pollyanna (Pollyanna, #1)",58040,https://www.goodreads.com/book/show/1000751.Po...,https://s.gr-assets.com/assets/nophoto/book/11...,pollyanna pollyanna 1,0.000069,0.000103
...,...,...,...,...,...,...,...,...,...,...
24154,9998,2,0.000000,The Woman in the Dunes,11841,https://www.goodreads.com/book/show/9998.The_W...,https://images.gr-assets.com/books/1361254930m...,the woman in the dunes,0.000338,0.000000
24155,9998825,1,4.000000,"Northwest Angle (Cork O'Connor, #11)",3357,https://www.goodreads.com/book/show/9998825-no...,https://s.gr-assets.com/assets/nophoto/book/11...,northwest angle cork oconnor 11,0.000298,0.001192
24156,9999,1,0.000000,The Box Man,2791,https://www.goodreads.com/book/show/9999.The_B...,https://images.gr-assets.com/books/1320459929m...,the box man,0.000358,0.000000
24157,9999107,2,1.000000,The American Heiress,24522,https://www.goodreads.com/book/show/9999107-th...,https://images.gr-assets.com/books/1307342832m...,the american heiress,0.000163,0.000163


In [102]:
my_books["mod_title"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex = True).str.lower()

In [103]:
my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\sushm\AppData\Local\Temp\ipykernel_14820\2213873673.py:1: SyntaxWarning: invalid escape sequence '\s'
  my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)


In [104]:
book_recs = book_recs[~book_recs["mod_title"].isin(my_books["mod_title"])]

In [105]:
book_recs = book_recs[book_recs["count"]>2]

In [106]:
book_recs = book_recs[book_recs["mean"]> 4]

In [107]:
top_recs = book_recs.sort_values("score", ascending=False)

In [83]:
top_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
2558,78983,4,4.250000,"Kane and Abel (Kane and Abel, #1)",75215,https://www.goodreads.com/book/show/78983.Kane...,https://s.gr-assets.com/assets/nophoto/book/11...,kane and abel kane and abel 1,0.000213,0.000904
1441,2767793,4,4.250000,"The Hero of Ages (Mistborn, #3)",149260,https://www.goodreads.com/book/show/2767793-th...,https://images.gr-assets.com/books/1480717763m...,the hero of ages mistborn 3,0.000107,0.000456
2260,62291,5,4.800000,"A Storm of Swords (A Song of Ice and Fire, #3)",477834,https://www.goodreads.com/book/show/62291.A_St...,https://images.gr-assets.com/books/1497931121m...,a storm of swords a song of ice and fire 3,0.000052,0.000251
1173,2318271,3,4.333333,The Last Lecture,245804,https://www.goodreads.com/book/show/2318271.Th...,https://images.gr-assets.com/books/1388075896m...,the last lecture,0.000037,0.000159
1100,22034,3,4.333333,The Godfather,259150,https://www.goodreads.com/book/show/22034.The_...,https://images.gr-assets.com/books/1394988109m...,the godfather,0.000035,0.000150
243,119322,4,4.250000,"The Golden Compass (His Dark Materials, #1)",973154,https://www.goodreads.com/book/show/119322.The...,https://images.gr-assets.com/books/1505766203m...,the golden compass his dark materials 1,0.000016,0.000070
1906,4381,3,4.333333,Fahrenheit 451,591506,https://www.goodreads.com/book/show/4381.Fahre...,https://images.gr-assets.com/books/1351643740m...,fahrenheit 451,0.000015,0.000066
600,157993,3,4.333333,The Little Prince,763309,https://www.goodreads.com/book/show/157993.The...,https://images.gr-assets.com/books/1367545443m...,the little prince,0.000012,0.000051


In [108]:
def make_clickable(val):
    return '<a target="_blank" href ="{}">Goodreads</a.'.format(val)

def show_image(val):
    return '<img src="{}" width=50></img>'.format(val)

In [109]:
top_recs.style.format({"url": make_clickable, "cover_image": show_image})

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
19021,62291,46,4.108696,"A Storm of Swords (A Song of Ice and Fire, #3)",477834,Goodreads</a.,,a storm of swords a song of ice and fire 3,0.004428,0.018195
8863,20755955,3,4.333333,The Accidental Prime Minister,2312,Goodreads</a.,,the accidental prime minister,0.003893,0.016869
4963,15803144,3,4.333333,The Emerald Mile: The Epic Story of the Fastest Ride in History Through the Heart of the Grand Canyon,2842,Goodreads</a.,,the emerald mile the epic story of the fastest ride in history through the heart of the grand canyon,0.003167,0.013723
0,1,69,4.405797,"Harry Potter and the Half-Blood Prince (Harry Potter, #6)",1713866,Goodreads</a.,,harry potter and the halfblood prince harry potter 6,0.002778,0.012239
18610,6,71,4.309859,"Harry Potter and the Goblet of Fire (Harry Potter, #4)",1792561,Goodreads</a.,,harry potter and the goblet of fire harry potter 4,0.002812,0.012120
3782,136251,69,4.376812,"Harry Potter and the Deathly Hallows (Harry Potter, #7)",1784684,Goodreads</a.,,harry potter and the deathly hallows harry potter 7,0.002668,0.011676
8523,2,70,4.100000,"Harry Potter and the Order of the Phoenix (Harry Potter, #5)",1766895,Goodreads</a.,,harry potter and the order of the phoenix harry potter 5,0.002773,0.011370
17214,4808763,5,5.000000,The Last Question,12166,Goodreads</a.,,the last question,0.002055,0.010275
19993,6748,7,4.571429,A Supposedly Fun Thing I'll Never Do Again: Essays and Arguments,23746,Goodreads</a.,,a supposedly fun thing ill never do again essays and arguments,0.002064,0.009433
1797,118252,3,4.333333,Mahabharata,4230,Goodreads</a.,,mahabharata,0.002128,0.009220
